In [0]:
import os
try:
  os.mkdir("data")
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1iUQc0KZ0Dit2iglPyEOF9q1dfNc4ERRp' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1iUQc0KZ0Dit2iglPyEOF9q1dfNc4ERRp" -O haha.zip && rm -rf /tmp/cookies.txt
  !unzip haha.zip
  !rm haha.zip
except:
  pass

--2020-05-16 19:08:25--  https://docs.google.com/uc?export=download&confirm=NCML&id=1iUQc0KZ0Dit2iglPyEOF9q1dfNc4ERRp
Resolving docs.google.com (docs.google.com)... 74.125.200.101, 74.125.200.138, 74.125.200.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.200.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-3s-docs.googleusercontent.com/docs/securesc/fjldcr3fv3jlsiband15gnmqroqf3u5n/lbvqnmosqb5h7vsqgrppnmrl7bptedbq/1589656050000/06685749744218960586/17380533698025392027Z/1iUQc0KZ0Dit2iglPyEOF9q1dfNc4ERRp?e=download [following]
--2020-05-16 19:08:25--  https://doc-0k-3s-docs.googleusercontent.com/docs/securesc/fjldcr3fv3jlsiband15gnmqroqf3u5n/lbvqnmosqb5h7vsqgrppnmrl7bptedbq/1589656050000/06685749744218960586/17380533698025392027Z/1iUQc0KZ0Dit2iglPyEOF9q1dfNc4ERRp?e=download
Resolving doc-0k-3s-docs.googleusercontent.com (doc-0k-3s-docs.googleusercontent.com)... 74.125.130.132, 2404:6800:4003:c01::84
Connecting

In [0]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

> This notebook aims to push the public LB under 0.50. Certainly, the competition is not yet at its peak and there clearly remains room for improvement.

# Credits

* [First R notebook](https://www.kaggle.com/kailex/m5-forecaster-v2)
* [Python translation](https://www.kaggle.com/kneroma/m5-forecast-v2-python)

In [0]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [0]:
pd.options.display.max_columns = 50

In [0]:
h = 28 
max_lags = 57
tr_last = 1913
fday = datetime(2016,4, 25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [0]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    prices = pd.read_csv("data/sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            #encode as integer 1-7, 1-12 so on
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()

    cal = pd.read_csv("data/calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"]) # to datetime format
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            #encode as integer 1-7, 1-12 so on
            cal[col] = cal[col].cat.codes.astype("int16") 
            cal[col] -= cal[col].min()


    #fugdy? ...
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    # which days to read from csv
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    # which categories to read
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    # ignore nros for now
    dt = pd.read_csv("data/sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)

    for col in catcols[1:]:
        dt[col] = dt[col].cat.codes.astype("int16")
        dt[col] -= dt[col].min()
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    # insanely nice function calls..    
    dt = pd.melt(dt,
        id_vars = catcols,
        value_vars = [col for col in dt.columns if col.startswith("d_")],
        var_name = "d",
        value_name = "sales")
    # merge nicely
    dt = dt.merge(cal, on= "d", copy = True)
    # merge nicely
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = True)
    
    return dt

In [0]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [0]:
FIRST_DAY = 750 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [0]:
%%time

df = create_dt(is_train=True, first_day= FIRST_DAY)
#print(df.shape)

CPU times: user 20.9 s, sys: 2.3 s, total: 23.2 s
Wall time: 23.5 s


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32673623 entries, 0 to 32673622
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
dtypes: datetime64[ns](1), float32(5), int16(14), object(2)
memory us

In [0]:
#%%time
#create_fea(df)
#df.shape

In [0]:
df.info()

In [0]:
df.head()

In [0]:
df.dropna(inplace = True)
df.shape

(32673623, 22)

In [0]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [0]:
# train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
# fake_valid_inds = np.random.choice(len(X_train), 1000000, replace = False)
# fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
#                              free_raw_data=False)   # This is just a subsample of the training set, not a real validation set !

In [0]:
%%time

#np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_00, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

CPU times: user 12.2 s, sys: 22.2 ms, total: 12.2 s
Wall time: 12.2 s


In [0]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

631

In [0]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 120,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}

In [0]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[20]	valid_0's rmse: 3.0174
[40]	valid_0's rmse: 2.8244
[60]	valid_0's rmse: 2.75415
[80]	valid_0's rmse: 2.72546
[100]	valid_0's rmse: 2.70571
[120]	valid_0's rmse: 2.68244
CPU times: user 25min 36s, sys: 1.81 s, total: 25min 38s
Wall time: 13min 26s


In [0]:
m_lgb.save_model("model.lgb")

In [0]:
%%time

alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv",index=False)

0 2016-04-25 00:00:00
1 2016-04-26 00:00:00


In [0]:
sub.head(10)

In [0]:
sub.head(10)

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.910026,0.847982,0.850890,0.801789,1.071719,1.296671,1.350886,1.009637,0.970509,0.995156,0.974088,1.114272,1.343018,1.194240,0.992385,0.929573,0.965841,0.963692,1.112975,1.385381,1.294539,0.993150,0.869451,0.849285,0.855230,1.022105,1.262562,1.233375
1,FOODS_1_001_CA_2_validation,0.940942,0.954666,0.882539,1.273918,1.290628,1.354433,1.554513,0.920688,0.926300,0.931731,0.960712,1.120798,1.493172,1.338206,1.020169,0.987166,1.002741,1.031221,1.208342,1.641095,1.501229,1.032052,0.982766,1.012417,1.052117,1.241158,1.689393,1.424921
2,FOODS_1_001_CA_3_validation,1.091795,1.044925,0.949970,0.917520,0.990185,1.107758,1.223882,1.105829,1.137468,1.004840,1.102019,1.126727,1.455431,1.267847,1.116070,1.112878,1.048881,1.069820,1.132724,1.674212,1.766982,1.147176,1.055445,0.980131,0.969123,1.054151,1.391111,1.266828
3,FOODS_1_001_CA_4_validation,0.414818,0.361572,0.355099,0.349345,0.405143,0.451183,0.517519,0.398493,0.426824,0.414165,0.423309,0.406780,0.444357,0.421732,0.376850,0.383107,0.405915,0.428294,0.457459,0.482851,0.500463,0.385219,0.363917,0.368164,0.373390,0.419760,0.457920,0.481183
4,FOODS_1_001_TX_1_validation,0.180371,0.179159,0.170700,0.171773,0.172805,0.180151,0.228228,0.476832,0.429286,0.425293,0.456359,0.464892,0.468873,0.397233,0.370246,0.397884,0.327414,0.328859,0.393753,0.380485,0.353399,0.295464,0.280412,0.275795,0.272560,0.278792,0.331847,0.318894
5,FOODS_1_001_TX_2_validation,0.482585,0.436249,0.445677,0.395896,0.447061,0.496326,0.577034,0.472376,0.479704,0.438499,0.524659,0.537258,0.614668,0.558372,0.494018,0.450336,0.495783,0.476332,0.553735,0.581341,0.591867,0.459162,0.425086,0.428849,0.435233,0.481264,0.547399,0.528644
6,FOODS_1_001_TX_3_validation,0.398578,0.368744,0.376498,0.413743,0.451219,0.457103,0.598738,0.482092,0.538495,0.483629,0.515115,0.534072,0.571413,0.489207,0.485516,0.451393,0.498438,0.498808,0.542449,0.567497,0.566333,0.453887,0.432316,0.435424,0.440499,0.471629,0.539260,0.506802
7,FOODS_1_001_WI_1_validation,0.331981,0.372827,0.364709,0.363885,0.447759,0.680631,0.649428,0.457657,0.437654,0.419107,0.512862,0.592773,0.799761,0.613822,0.536360,0.525176,0.572492,0.578808,0.714459,0.864306,0.765611,0.558961,0.541106,0.556651,0.567618,0.703742,1.026890,0.805103
8,FOODS_1_001_WI_2_validation,0.310635,0.340609,0.353715,0.338573,0.414242,0.423919,0.425699,0.437361,0.506364,0.413411,0.451346,0.507607,0.515267,0.403375,0.435103,0.417913,0.494655,0.484010,0.488920,0.529035,0.497015,0.364418,0.391413,0.381275,0.383275,0.403016,0.440113,0.405239
9,FOODS_1_001_WI_3_validation,0.237874,0.235062,0.209730,0.220884,0.267701,0.319444,0.379183,0.366106,0.388815,0.348863,0.381240,0.436294,0.422055,0.371011,0.346316,0.336610,0.364897,0.351345,0.413273,0.463644,0.426028,0.320039,0.311917,0.324671,0.319644,0.378388,0.424405,0.364692


In [0]:
sub.id.nunique(), sub["id"].str.contains("validation$").sum()

(60980, 30490)

In [0]:
sub.shape

(60980, 29)